# Cutout Regularization — Temelden Derine (CIFAR-100 / CNN)

Bu notebook bir **ders notu + pratik demo** gibidir:
- Cutout nedir, neden çalışır?
- Hangi problemi çözer? (overfitting, occlusion-robustness)
- Diğer regularization/augmentation tekniklerinden farkları
- PyTorch implementasyonları (Cutout, RandomErasing, DropBlock vs. karşılaştırma)
- CIFAR-100 pipeline içine entegrasyon örnekleri

> Not: Bu notebook **internet gerektirmez** (örnek görselleştirmeler random tensor ile).  
> CIFAR-100 ile gerçek eğitim yapmak istersen `torchvision.datasets.CIFAR100(download=True)` internet ister.


In [ ]:
# === 0) Imports ===
import math
import random
from dataclasses import dataclass
from typing import Tuple, Optional, Dict, Any

import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt


## 1) Regularization Nedir? (Çok kısa ama net)

CNN'ler genelde **fazla güçlü** modeller. Küçük/orta veriyle eğitirken:
- Train performansı ↑
- Test performansı aynı oranda artmaz → **overfit**

**Regularization** = modelin "ezber" yerine "genelleme" öğrenmesini zorlamak için eklenen baskılar.

Büyük resim:
- **Model içi regularization**: weight decay (L2), dropout, DropPath, DropBlock, label smoothing...
- **Veri tabanlı regularization (augmentation)**: random crop, flip, color jitter, Mixup, CutMix, Cutout...
- **Optimizasyon tabanlı**: early stopping, SWA, stochasticity (SGD), etc.

Cutout → **augmentation tabanlı regularization**.


## 2) Cutout Nedir? (En temel tanım)

**Cutout**: Eğitim sırasında giriş görüntüsünde rastgele bir **kare (rectangular) alanı** seçip o alanı "silmek".

- Silmek = çoğu implementasyonda piksel değerlerini **0** yapmak
- Alternatif: görüntü ortalaması ile doldurmak (mean fill)

**Amaç**: Modeli tek bir lokal ipucuna aşırı bağımlı olmaktan kurtarmak.

Örnek düşün:
- Model hep "kedinin gözü"nden sınıfı çözüyorsa
- Cutout o bölgeyi kapatınca model diğer ipuçlarını da öğrenmek zorunda kalır.


### 2.1 Cutout'un sezgisel etkisi

Cutout şunları zorlar:
- **Part-based** öğrenme (parça bazlı özellikler)
- **Context-based** öğrenme (bağlam)
- **Occlusion robustness** (objenin bir kısmı kapalıyken bile tanıma)

Dolayısıyla overfitting'i kırar ve testte daha stabil olur.


## 3) Matematiksel bakış (kısa ama derin)

Girdi görüntü: \(x \in \mathbb{R}^{C \times H \times W}\)

Cutout bir maske uygular:
- \(m \in \{0,1\}^{H \times W}\)
- Kesilen bölgede 0, diğer bölgede 1

Uygulama:
\[ x' = x \odot m \]

Bu, eğitim sırasında **girdi dağılımını** genişletir:
- Model artık sadece "tam ve temiz" görüntü görmez
- Görüntünün rastgele eksik versiyonlarını da görür

Bu yüzden bir tür **data-driven regularization** olur.


## 4) Cutout ile Dropout/DropBlock/DropPath farkı

**En kritik ayrım:** Cutout **input üzerinde** çalışır.

| Teknik | Nerede uygulanır? | Ne düşürür? | Tipik etki |
|---|---|---|---|
| Dropout | activations | tekil nöron/eleman | FC veya bazı CNN katmanlarında |
| SpatialDropout | activations | kanal | kanal bazlı genelleme |
| DropBlock | activations | uzamsal blok | feature map üzerinde "cutout benzeri" |
| DropPath / Stochastic Depth | residual branch | tüm yol | çok derin ağlarda genelleme |
| **Cutout** | **input image** | **piksel bloğu** | occlusion + overfit kırma |
| Mixup | input+label | iki örneği karıştırır | decision boundary yumuşatır |
| CutMix | input+label | patch değiş tokuş | localization + robustness |

Özet:  
- Cutout ≈ “input-level DropBlock” gibi düşünülebilir  
- Ama Cutout label'i değiştirmez (Mixup/CutMix gibi değil)


## 5) Cutout nerede iyi çalışır, nerede riskli?

**İyi çalıştığı yerler:**
- CIFAR-10/100 gibi küçük görüntüler (32×32)
- Overfit eğilimi olan modeller
- Obje kısmi kapanması olası senaryolar

**Riskler:**
- Cutout boyutu çok büyükse bilgi aşırı kaybolur → öğrenme zorlaşır
- Çok agresif cutout + ağır augmentation birleşirse underfit yapabilir
- Bazı görevlerde (ör. medikal görüntü) kritik bölgeyi silmek performansı düşürebilir

**Tipik hiperparametreler (CIFAR):**
- length/size: 8–16 (32x32 için)
- prob: 0.5–1.0 arası uygulanma olasılığı


## 6) PyTorch: Cutout Implementasyonu (Tensor transform)

Aşağıdaki implementasyon **ToTensor() sonrası** çalışır:  
- input: `torch.Tensor [C,H,W]`
- seçilen alanı 0'lar
- opsiyonel: mean ile doldurma


In [ ]:
# === 6) Cutout Transform ===
class Cutout:
    def __init__(self, size: int = 8, p: float = 1.0, fill: str = "zero", mean: Tuple[float,float,float]=(0.0,0.0,0.0)):
        if size < 1:
            raise ValueError("size >= 1 olmalı")
        if not (0.0 <= p <= 1.0):
            raise ValueError("p 0..1 olmalı")
        if fill not in ["zero", "mean"]:
            raise ValueError("fill 'zero' veya 'mean' olmalı")
        self.size = int(size)
        self.p = float(p)
        self.fill = fill
        self.mean = mean

    def __call__(self, x: torch.Tensor) -> torch.Tensor:
        # x: [C,H,W]
        if (self.p == 0.0) or (random.random() > self.p):
            return x
        if x.dim() != 3:
            raise ValueError(f"Cutout [C,H,W] bekler. Geldi: {tuple(x.shape)}")

        c, h, w = x.shape
        s = self.size

        cy = random.randint(0, h - 1)
        cx = random.randint(0, w - 1)

        y1 = max(0, cy - s // 2)
        y2 = min(h, cy + (s - s // 2))
        x1 = max(0, cx - s // 2)
        x2 = min(w, cx + (s - s // 2))

        if self.fill == "zero":
            x[:, y1:y2, x1:x2] = 0.0
        else:
            mean = torch.tensor(self.mean, device=x.device, dtype=x.dtype).view(3,1,1) if c == 3 else torch.zeros((c,1,1), device=x.device, dtype=x.dtype)
            x[:, y1:y2, x1:x2] = mean
        return x


## 7) Hızlı görselleştirme: Cutout ne yapıyor?

Aşağıda random bir “görüntü” üzerinde Cutout’u görselleştiriyoruz.  
Gerçek CIFAR görüntüsü değil; sadece mantığı görmen için.


In [ ]:
# === 7) Demo visualization (random image) ===
def show_tensor_img(x: torch.Tensor, title: str = ""):
    # x: [C,H,W], assume 0..1
    img = x.detach().cpu().permute(1,2,0).clamp(0,1).numpy()
    plt.figure()
    plt.imshow(img)
    plt.axis("off")
    plt.title(title)
    plt.show()

torch.manual_seed(0)
img = torch.rand(3, 32, 32)

cut = Cutout(size=12, p=1.0, fill="zero")
img2 = img.clone()
img2 = cut(img2)

show_tensor_img(img, "Orijinal (random)")
show_tensor_img(img2, "Cutout uygulanmış (size=12)")


## 8) RandomErasing ile ilişki

PyTorch `torchvision.transforms.RandomErasing` var. O da Cutout'a benzer:
- RandomErasing genelde **rectangle** siler
- fill değerini seçebilirsin
- Uygulama olasılığı var

Fark: Cutout literatürde daha basit/standart olarak kare blokla anılır.
Pratikte ikisi aynı aile.


In [ ]:
# === 8) RandomErasing örneği (Tensor üstünde) ===
# Not: RandomErasing transform'u genelde ToTensor sonrası kullanılır.
try:
    import torchvision.transforms as T
    re = T.RandomErasing(p=1.0, scale=(0.05, 0.05), ratio=(1.0, 1.0), value=0)
    img3 = img.clone()
    img3 = re(img3)
    show_tensor_img(img3, "torchvision RandomErasing (yakın akraba)")
except Exception as e:
    print("torchvision yok veya import sorunu:", e)


## 9) DropBlock ile konsept farkı (neden ikisini karıştırmayalım?)

DropBlock:
- Input'ta değil, **feature map** üzerinde
- CNN'in ara katmanlarında "aktivasyon bloklarını" düşürür
- Genelde daha derin katmanlarda daha faydalı (yüksek seviyeli feature'larda)

Cutout:
- Input'ta çalışır
- Model daha başlangıçta occlusion'ı görür

Bu yüzden:
- Cutout tek başına güçlü olabilir
- DropBlock tek başına güçlü olabilir
- İkisini aynı anda kullanmak bazen iyi, bazen gereksiz agresif olabilir.


## 10) CIFAR-100 pipeline içine nasıl eklenir?

Tipik sıra:
1) RandomCrop / Flip (PIL veya Tensor)  
2) ToTensor  
3) Cutout (Tensor)  
4) Normalize

Aşağıdaki hücre sadece örnek şablon.


In [ ]:
# === 10) CIFAR-100 transform şablonu ===
# Bu hücreyi gerçek eğitime bağlamak istersen uncomment et.
#
# import torchvision
# import torchvision.transforms as T
#
# mean = (0.5071, 0.4867, 0.4408)
# std  = (0.2675, 0.2565, 0.2761)
#
# train_tf = T.Compose([
#     T.RandomCrop(32, padding=4),
#     T.RandomHorizontalFlip(),
#     T.ToTensor(),
#     Cutout(size=12, p=1.0, fill="zero"),
#     T.Normalize(mean, std),
# ])
#
# test_tf = T.Compose([
#     T.ToTensor(),
#     T.Normalize(mean, std),
# ])


## 11) “Derin” taraf: Cutout neden genelleme sağlıyor?

Cutout, eğitimde şu iki şeyi aynı anda yapar:
1) **Input variances** artırır: modelin gördüğü örnek uzayını genişletir
2) **Inductive bias** ekler: “Her zaman her parça görünmeyebilir” varsayımını zorla öğretir

Bunu bir çeşit “occlusion prior” gibi düşünebilirsin.

**Decision boundary** açısından:
- Model tek bir patch'e aşırı duyarlı olursa boundary keskinleşir ve overfit artar
- Cutout bu aşırı duyarlılığı kırar → daha smooth boundary

Bu, Dropout'un FC katmanlarda yaptığı şeye benzer, ama **input düzeyinde**.


## 12) Hiperparametre tuning rehberi (pratik)

CIFAR-100 (32×32) için pratik aralık:
- `size`: 8, 12, 16
- `p`: 0.5 → 1.0
- `fill`: zero genelde OK (Normalize sonrası mean=0'a yakın)
- Çok ağır augment (Mixup/CutMix) varsa Cutout'u azalt veya kapat

Basit tarama:
- Baseline
- Cutout(size=8)
- Cutout(size=12)
- Cutout(size=16)


## 13) Kısa özet

- Cutout = giriş görüntüsünde rastgele bir alanı silerek **augmentation tabanlı regularization** sağlar.
- DropBlock'a benzer ama **input-level**.
- Mixup/CutMix gibi label karıştırmaz.
- Küçük görüntülerde (CIFAR) çok etkili olabilir.
